In [13]:
MLFLOW_URL = "http://localhost:5000"
AWS_ACCESS_KEY_ID="minio123"
AWS_SECRET_ACCESS_KEY="minio123"
MLFLOW_S3_ENDPOINT_URL="http://localhost:9990"
BASE_AIRFLOW_DAG_RUN_ID = None

In [14]:
from os import environ

environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
environ["MLFLOW_S3_ENDPOINT_URL"] = MLFLOW_S3_ENDPOINT_URL
environ["MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR"] = "false"

In [15]:
import os
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import mlflow 
import logging

os.environ['import numpy as npTF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel(logging.ERROR)

In [16]:
import mlflow

if MLFLOW_URL is not None:
    environ["MLFLOW_S3_ENDPOINT_URL"] = MLFLOW_S3_ENDPOINT_URL
    mlflow.set_tracking_uri(MLFLOW_URL)

In [17]:
META_MODEL_WITNESSES = [
    "CHSH_OPTIMAL",
    "CONCURRENCE",
    "ENTROPY",
    "NEGATIVITY",
    "PPT"
]

RUN_IDS = {}

for witness in META_MODEL_WITNESSES:
    client = mlflow.tracking.MlflowClient()

    experiments = client.search_experiments(
        filter_string = "name = 'ML Quantum Entanglement'"
    )
    experiment_id = experiments[0].experiment_id

    filter_string = (
        f"tags.airflow_dag_run_id = '{BASE_AIRFLOW_DAG_RUN_ID}-{witness}' and "
        f"tags.pipeline_step = 'training'"
    )   
    runs = client.search_runs(
        experiment_ids=[experiment_id], 
        filter_string=filter_string
    )
    run_id = runs[0].info.run_id
    RUN_IDS[witness] = run_id

In [18]:
all_models = {}

for witness_name, run_id in RUN_IDS.items():
    model_uri = f"runs:/{run_id}/model"
    all_models[witness_name] = mlflow.tensorflow.load_model(model_uri)

In [19]:

def get_meta_input(models, data):
    meta_input = []
    for model in models:
        predictions = model.predict(data, verbose=0)
        meta_input.append(predictions)
    return np.concatenate(meta_input, axis=1)

all_models_array = all_models.values()

In [20]:
def get_PPT_simulation_data_path():
    filename = "simulation-PPT.npz"
    if BASE_AIRFLOW_DAG_RUN_ID is None:
        data_file_path = f"./simulated_data/{filename}.npz"
    else:
        client = mlflow.tracking.MlflowClient()

        experiments = client.search_experiments(
            filter_string = "name = 'ML Quantum Entanglement'"
        )
        experiment_id = experiments[0].experiment_id

        runs = client.search_runs(
            experiment_ids=[experiment_id], 
            filter_string=f"tags.airflow_dag_run_id = '{BASE_AIRFLOW_DAG_RUN_ID}-PPT' and tags.pipeline_step = 'simulation'"
        )
        run_id = runs[0].info.run_id
        local_path = mlflow.artifacts.download_artifacts(run_id=run_id, artifact_path="simulated_data")
        data_file_path = os.path.join(local_path, filename)
    return data_file_path

data = np.load(get_PPT_simulation_data_path())
simulated_states = data["states"]
simulated_labels = data["labels"]

In [21]:
X_train, X_val, y_train, y_val = train_test_split(simulated_states, simulated_labels, test_size=0.2, random_state=42)

# Assuming each model outputs a single probability value
meta_input_shape = len(all_models_array)
meta_input = Input(shape=(meta_input_shape,))

# Define a simple feedforward neural network
x = Dense(10, activation='relu')(meta_input)
x = Dense(10, activation='relu')(x)
meta_output = Dense(1, activation='sigmoid')(x)

meta_model = Model(inputs=meta_input, outputs=meta_output)
meta_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Prepare training data
# Assuming you have your training data `X_train` and labels `y_train`
X_train_meta = get_meta_input(all_models_array, X_train)
y_train_meta = y_train

# Train the meta-model
meta_model.fit(X_train_meta, y_train_meta, epochs=10, batch_size=32, validation_split=0.2, verbose=0)


In [23]:
from simulation_utils import create_random_bell_phi, \
    create_random_bell_psi, create_random_separable, \
        create_entangled_werner_state_with_chsh_violation, \
            create_entangled_werner_state_without_chsh_violation, \
                create_not_entangled_werner_state_dm, \
                    create_mixed_not_entangled_state, flatten_density_matrix

sim_functions = [
    create_random_bell_phi, 
    create_random_bell_psi,
    create_random_separable, 
    create_entangled_werner_state_with_chsh_violation, 
    create_entangled_werner_state_without_chsh_violation, 
    create_not_entangled_werner_state_dm,
    create_mixed_not_entangled_state
]

for sim_func in sim_functions:
    print(f"Simulation function: {sim_func.__name__}")
    state_input = tf.constant([flatten_density_matrix(sim_func())])

    for witness_name, model in all_models.items():
        prediction = model.predict(state_input, verbose=0)
        print(f"Witness model {witness_name} entanglement prediction {prediction}")
    
    print("---")
    state_meta_input = get_meta_input(all_models_array, state_input)
    prediction = meta_model.predict(state_meta_input, verbose=0)
    print(f"Meta model entanglement prediction {prediction}\n")  

Simulation function: create_random_bell_phi
Witness model CHSH_OPTIMAL entanglement prediction [[0.99965763]]
Witness model CONCURRENCE entanglement prediction [[0.99962896]]
Witness model ENTROPY entanglement prediction [[0.9997985]]
Witness model NEGATIVY entanglement prediction [[0.99990594]]
Witness model PPT entanglement prediction [[0.9999844]]
---
Meta model entanglement prediction [[0.9985204]]

Simulation function: create_random_bell_psi
Witness model CHSH_OPTIMAL entanglement prediction [[0.9976435]]
Witness model CONCURRENCE entanglement prediction [[0.9987242]]
Witness model ENTROPY entanglement prediction [[0.9984875]]
Witness model NEGATIVY entanglement prediction [[0.99998933]]
Witness model PPT entanglement prediction [[0.99997234]]
---
Meta model entanglement prediction [[0.99851215]]

Simulation function: create_random_separable
Witness model CHSH_OPTIMAL entanglement prediction [[0.00044619]]
Witness model CONCURRENCE entanglement prediction [[2.0733685e-07]]
Witness